In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 47.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2ef0b618564c7bdbf5a9751614c553c96af081d0013756b21dba1d7b0d0a950c
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
information_df = spark.read.csv('informations_households.csv', header = True, inferSchema= True)
holidays_df = spark.read.csv('uk_bank_holidays.csv', header = True, inferSchema= True)
weather_daily_df = spark.read.csv('weather_daily_darksky.csv', header = True, inferSchema= True)
weather_hourly_df = spark.read.csv('weather_hourly_darksky.csv', header = True, inferSchema= True)
daily_df = spark.read.csv('daily_dataset.csv', header = True, inferSchema= True)
acorn_df = spark.read.csv('acorn_details.csv', header = True, inferSchema= True)

In [5]:
information_df.createOrReplaceTempView('information_df')
holidays_df.createOrReplaceTempView('holidays_df')
weather_daily_df.createOrReplaceTempView('weather_daily_df')
weather_hourly_df.createOrReplaceTempView('weather_hourly_df')
daily_df.createOrReplaceTempView('daily_df')
acorn_df.createOrReplaceTempView('acorn_df')

In [6]:
spark.sql(''' SELECT *
              FROM information_df
              LIMIT 5 ''').show()

+---------+--------+-------+-------------+-------+
|    LCLid|stdorToU|  Acorn|Acorn_grouped|   file|
+---------+--------+-------+-------------+-------+
|MAC005492|     ToU| ACORN-|       ACORN-|block_0|
|MAC001074|     ToU| ACORN-|       ACORN-|block_0|
|MAC000002|     Std|ACORN-A|     Affluent|block_0|
|MAC003613|     Std|ACORN-A|     Affluent|block_0|
|MAC003597|     Std|ACORN-A|     Affluent|block_0|
+---------+--------+-------+-------------+-------+



In [7]:
spark.sql(''' SELECT *
              FROM holidays_df
              ''').show()

+-------------------+--------------------+
|      Bank holidays|                Type|
+-------------------+--------------------+
|2012-12-26 00:00:00|          Boxing Day|
|2012-12-25 00:00:00|       Christmas Day|
|2012-08-27 00:00:00| Summer bank holiday|
|2012-05-06 00:00:00|Queen?s Diamond J...|
|2012-04-06 00:00:00|Spring bank holid...|
|2012-07-05 00:00:00|Early May bank ho...|
|2012-09-04 00:00:00|       Easter Monday|
|2012-06-04 00:00:00|         Good Friday|
|2012-02-01 00:00:00|New Year?s Day (s...|
|2013-12-26 00:00:00|          Boxing Day|
|2013-12-25 00:00:00|       Christmas Day|
|2013-08-26 00:00:00| Summer bank holiday|
|2013-05-27 00:00:00| Spring bank holiday|
|2013-06-05 00:00:00|Early May bank ho...|
|2013-01-04 00:00:00|       Easter Monday|
|2013-03-29 00:00:00|         Good Friday|
|2013-01-01 00:00:00|      New Year?s Day|
|2014-12-26 00:00:00|          Boxing Day|
|2014-12-25 00:00:00|       Christmas Day|
|2014-08-25 00:00:00| Summer bank holiday|
+----------

In [8]:
spark.sql(''' SELECT *
              FROM daily_df
              LIMIT 5 ''').show()

+---------+-------------------+-------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|    LCLid|                day|energy_median|        energy_mean|        energy_max|energy_count|         energy_std|        energy_sum|         energy_min|
+---------+-------------------+-------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|MAC000131|2011-12-15 00:00:00|        0.485|0.43204545454545457|             0.868|          22|0.23914579678767536|             9.505|0.07200000000000001|
|MAC000131|2011-12-16 00:00:00|       0.1415|0.29616666875000003|         1.1160001|          48| 0.2814713178628203|14.216000100000002|              0.031|
|MAC000131|2011-12-17 00:00:00|       0.1015|          0.1898125|             0.685|          48| 0.1884046862418033|             9.111|              0.064|
|MAC000131|2011-12-18 00:00:00|        0.114| 0.2189791666

In [9]:
spark.sql(''' SELECT *
              FROM daily_df
              INNER JOIN information_df
              ON daily_df.LCLid = information_df.LCLid
              ''').show()

+---------+-------------------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+---------+--------+-------+-------------+--------+
|    LCLid|                day|      energy_median|        energy_mean|        energy_max|energy_count|         energy_std|        energy_sum|         energy_min|    LCLid|stdorToU|  Acorn|Acorn_grouped|    file|
+---------+-------------------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+---------+--------+-------+-------------+--------+
|MAC000131|2011-12-15 00:00:00|              0.485|0.43204545454545457|             0.868|          22|0.23914579678767536|             9.505|0.07200000000000001|MAC000131|     Std|ACORN-E|     Affluent|block_22|
|MAC000131|2011-12-16 00:00:00|             0.1415|0.29616666875000003|         1.1160001|          48| 0.2814713178628203|14.216000100000002|      

In [10]:
spark.sql('''SELECT *
              FROM daily_df
              ''').show()

+---------+-------------------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|    LCLid|                day|      energy_median|        energy_mean|        energy_max|energy_count|         energy_std|        energy_sum|         energy_min|
+---------+-------------------+-------------------+-------------------+------------------+------------+-------------------+------------------+-------------------+
|MAC000131|2011-12-15 00:00:00|              0.485|0.43204545454545457|             0.868|          22|0.23914579678767536|             9.505|0.07200000000000001|
|MAC000131|2011-12-16 00:00:00|             0.1415|0.29616666875000003|         1.1160001|          48| 0.2814713178628203|14.216000100000002|              0.031|
|MAC000131|2011-12-17 00:00:00|             0.1015|          0.1898125|             0.685|          48| 0.1884046862418033|             9.111|              0.064|
|MAC000131|2011-12-18 

**blockwise Maximum and MInimum energy consumption** 

In [11]:
spark.sql(''' SELECT information_df.file, 
                     SUM(daily_df.energy_max) AS MAX_Energy_Consumption,
                     SUM(daily_df.energy_min) AS MIN_Energy_Consumption
              FROM daily_df
              JOIN information_df
              ON daily_df.LCLid = information_df.LCLid 
              GROUP BY information_df.file
              ORDER BY information_df.file 
              LIMIT 5 ''').show()

+--------+----------------------+----------------------+
|    file|MAX_Energy_Consumption|MIN_Energy_Consumption|
+--------+----------------------+----------------------+
|block_22|    30823.649006200252|     2287.911000099982|
|block_35|      4524.29200350002|    431.51099999999394|
|block_36|    26950.763998700033|    1964.7630000999916|
|block_52|    24364.897002100108|     1831.367999999928|
|block_54|    26307.732002000055|     1709.884999899973|
+--------+----------------------+----------------------+



**blockwise Maximum and MInimum energy consumption at particular date**

In [12]:
spark.sql(''' SELECT information_df.file, date(daily_df.day),
                     SUM(daily_df.energy_max) AS MAX_Energy_Consumption,
                     SUM(daily_df.energy_min) AS MIN_Energy_Consumption
              FROM daily_df
              JOIN information_df
              ON daily_df.LCLid = information_df.LCLid
              WHERE daily_df.day = '2011-12-15 00:00:00'
              GROUP BY information_df.file, daily_df.day
              ORDER BY information_df.file
              LIMIT 5''').show()

+--------+----------+----------------------+----------------------+
|    file|       day|MAX_Energy_Consumption|MIN_Energy_Consumption|
+--------+----------+----------------------+----------------------+
|block_22|2011-12-15|     9.065999999999999|                 0.386|
|block_35|2011-12-15|     5.967000100000001|                 0.354|
|block_36|2011-12-15|             6.9920002|                 0.518|
|block_52|2011-12-15|                  0.75|                  0.11|
|block_54|2011-12-15|             4.1379999|                 0.146|
+--------+----------+----------------------+----------------------+



**AVG MAX and MIN energy consumption of all block**

In [13]:
sql_1 = spark.sql(''' SELECT AVG(MAX_Energy_Consumption) AS AVG_Energy_Consumption_MAX,
                     AVG(MIN_Energy_Consumption) AS AVG_Energy_Consumption_MIN
              FROM (
                     SELECT information_df.file, 
                     SUM(daily_df.energy_max) AS MAX_Energy_Consumption,
                     SUM(daily_df.energy_min) AS MIN_Energy_Consumption
              FROM daily_df
              JOIN information_df
              ON daily_df.LCLid = information_df.LCLid 
              GROUP BY information_df.file
              ORDER BY information_df.file 
                  ) AS f
               ''')

sql_1.createOrReplaceTempView('sql_1')
sql_1.show()

+--------------------------+--------------------------+
|AVG_Energy_Consumption_MAX|AVG_Energy_Consumption_MIN|
+--------------------------+--------------------------+
|        23234.184546063654|        1522.6660909636141|
+--------------------------+--------------------------+



**block that energy consumption more or less than avg**

In [14]:
sql_2 = spark.sql('''SELECT information_df.file, 
                     SUM(daily_df.energy_max) AS MAX_Energy_Consumption,
                     SUM(daily_df.energy_min) AS MIN_Energy_Consumption
              FROM daily_df
              JOIN information_df
              ON daily_df.LCLid = information_df.LCLid 
              GROUP BY information_df.file
              ORDER BY information_df.file''')

sql_2.createOrReplaceTempView('sql_2')

spark.sql(''' SELECT *,
              CASE
                WHEN sql_2.MAX_Energy_Consumption > (AVG(sql_2.MAX_Energy_Consumption)OVER()) THEN 'Greater than AVG'
                ELSE 'Less than AVG'
              END AS Status
              FROM sql_2 
              LIMIT 5''').show()

+--------+----------------------+----------------------+----------------+
|    file|MAX_Energy_Consumption|MIN_Energy_Consumption|          Status|
+--------+----------------------+----------------------+----------------+
|block_22|    30823.649006200252|     2287.911000099982|Greater than AVG|
|block_35|      4524.29200350002|    431.51099999999394|   Less than AVG|
|block_36|    26950.763998700033|    1964.7630000999916|Greater than AVG|
|block_52|    24364.897002100108|     1831.367999999928|Greater than AVG|
|block_54|    26307.732002000055|     1709.884999899973|Greater than AVG|
+--------+----------------------+----------------------+----------------+



**percentage energy consumption of blocks**

In [15]:
spark.sql(''' SELECT *, MAX_Energy_Consumption / SUM(MAX_Energy_Consumption)OVER() *100 AS MAX_Percentage_Consumption
              FROM sql_2
              LIMIT 5 ''').show()

+--------+----------------------+----------------------+--------------------------+
|    file|MAX_Energy_Consumption|MIN_Energy_Consumption|MAX_Percentage_Consumption|
+--------+----------------------+----------------------+--------------------------+
|block_22|    30823.649006200252|     2287.911000099982|        12.060461618952363|
|block_35|      4524.29200350002|    431.51099999999394|         1.770233305283524|
|block_36|    26950.763998700033|    1964.7630000999916|        10.545106283243186|
|block_52|    24364.897002100108|     1831.367999999928|         9.533326345769341|
|block_54|    26307.732002000055|     1709.884999899973|        10.293505224770245|
+--------+----------------------+----------------------+--------------------------+



**MAX and MIN energy consumption on bank holidays**

In [16]:
spark.sql(''' SELECT
                    
                    information_df.file, daily_df.day,
                    SUM(daily_df.energy_max) AS MAX_Energy_Consumption,
                    SUM(daily_df.energy_min) AS MIN_Energy_Consumption
                FROM holidays_df
                JOIN daily_df
                  ON holidays_df.`Bank holidays` = daily_df.day
                JOIN information_df
                  ON daily_df.LCLid = information_df.LCLid
            GROUP BY information_df.file, daily_df.day
            ORDER BY information_df.file, daily_df.day
               ''').show()

+--------+-------------------+----------------------+----------------------+
|    file|                day|MAX_Energy_Consumption|MIN_Energy_Consumption|
+--------+-------------------+----------------------+----------------------+
|block_22|2012-02-01 00:00:00|                27.147|                 2.332|
|block_22|2012-04-06 00:00:00|    22.639000200000005|                 2.002|
|block_22|2012-05-06 00:00:00|    28.678000100000006|    1.9869999999999994|
|block_22|2012-06-04 00:00:00|    21.919999999999995|                 1.678|
|block_22|2012-07-05 00:00:00|                18.859|                  1.49|
|block_22|2012-08-27 00:00:00|     35.67999999999999|                 2.966|
|block_22|2012-09-04 00:00:00|                35.575|                 2.908|
|block_22|2012-12-25 00:00:00|    36.676999900000006|                 3.723|
|block_22|2012-12-26 00:00:00|    36.674000199999995|    3.6860000000000004|
|block_22|2013-01-01 00:00:00|    45.132000500000004|    3.6629999999999994|

**MAX and MIN energy consumption on other than bank holidays**

In [17]:
spark.sql('''SELECT 
                    information_df.file, daily_df.day,
                    SUM(daily_df.energy_max) AS MAX_Energy_Consumption,
                    SUM(daily_df.energy_min) AS MIN_Energy_Consumption
                FROM holidays_df
                FULL JOIN daily_df
                  ON holidays_df.`Bank holidays` = daily_df.day
                JOIN information_df
                  ON daily_df.LCLid = information_df.LCLid
               WHERE holidays_df.`Bank holidays` IS NULL
            GROUP BY information_df.file, daily_df.day
            ORDER BY information_df.file, daily_df.day
              
               ''').show()

+--------+-------------------+----------------------+----------------------+
|    file|                day|MAX_Energy_Consumption|MIN_Energy_Consumption|
+--------+-------------------+----------------------+----------------------+
|block_22|2011-12-01 00:00:00|             6.1259999|                  0.44|
|block_22|2011-12-02 00:00:00|                 4.193|   0.31100000000000005|
|block_22|2011-12-03 00:00:00|     4.851999999999999|                 0.246|
|block_22|2011-12-04 00:00:00|                 4.768|                 0.259|
|block_22|2011-12-05 00:00:00|             4.7189999|   0.23700000000000002|
|block_22|2011-12-06 00:00:00|                  3.71|   0.23500000000000001|
|block_22|2011-12-07 00:00:00|                 3.327|   0.21300000000000002|
|block_22|2011-12-08 00:00:00|    4.2850000999999995|   0.29500000000000004|
|block_22|2011-12-09 00:00:00|    3.7210000999999995|                 0.269|
|block_22|2011-12-10 00:00:00|    3.4830000000000005|                 0.241|

In [18]:
spark.sql(''' SELECT
                     energy_count,
                     RANK() OVER(ORDER BY energy_count) AS rank_no
                FROM daily_df
               GROUP BY energy_count''').show()

+------------+-------+
|energy_count|rank_no|
+------------+-------+
|           0|      1|
|           1|      2|
|           9|      3|
|          18|      4|
|          19|      5|
|          20|      6|
|          21|      7|
|          22|      8|
|          23|      9|
|          24|     10|
|          25|     11|
|          26|     12|
|          27|     13|
|          28|     14|
|          29|     15|
|          30|     16|
|          31|     17|
|          32|     18|
|          33|     19|
|          34|     20|
+------------+-------+
only showing top 20 rows



In [19]:
spark.sql(''' SELECT 
                    LCLid, energy_max,
                    YEAR(day) AS year
              FROM daily_df
                      ''').show()

+---------+------------------+----+
|    LCLid|        energy_max|year|
+---------+------------------+----+
|MAC000131|             0.868|2011|
|MAC000131|         1.1160001|2011|
|MAC000131|             0.685|2011|
|MAC000131|0.6759999999999999|2011|
|MAC000131|0.7879999999999999|2011|
|MAC000131|             1.077|2011|
|MAC000131|             0.705|2011|
|MAC000131|             1.094|2011|
|MAC000131|0.7490000000000001|2011|
|MAC000131|             0.613|2011|
|MAC000131|             0.866|2011|
|MAC000131|             0.838|2011|
|MAC000131|             0.778|2011|
|MAC000131|             1.207|2011|
|MAC000131|             0.888|2011|
|MAC000131|             0.782|2011|
|MAC000131|             1.252|2011|
|MAC000131|             0.812|2012|
|MAC000131|             0.851|2012|
|MAC000131|             0.674|2012|
+---------+------------------+----+
only showing top 20 rows



In [20]:
spark.sql('''SELECT 
                  information_df.file,
                  YEAR(daily_df.day) AS year,
                  SUM(daily_df.energy_max) AS MAX_Energy_Consumption
              FROM daily_df
              INNER JOIN information_df
              ON daily_df.LCLid = information_df.LCLid
              GROUP BY
                  information_df.file,
                  year
              ORDER BY
                  information_df.file,
                  year 
              ''').show()

+--------+----+----------------------+
|    file|year|MAX_Energy_Consumption|
+--------+----+----------------------+
|block_22|2011|    346.56800120000014|
|block_22|2012|    12380.234003999913|
|block_22|2013|    15652.722000600028|
|block_22|2014|     2444.125000400005|
|block_35|2011|    126.26499970000003|
|block_35|2012|     2229.221001599999|
|block_35|2013|     1871.591001900004|
|block_35|2014|     297.2150002999998|
|block_36|2011|    197.44900089999993|
|block_36|2012|    10185.997996999997|
|block_36|2013|    14123.765999900002|
|block_36|2014|     2443.551000899998|
|block_52|2011|    34.078000100000004|
|block_52|2012|     8567.932999699973|
|block_52|2013|    13567.974003399986|
|block_52|2014|     2194.911998900001|
|block_54|2011|     76.73699969999998|
|block_54|2012|     9122.000000799984|
|block_54|2013|     14649.40600229996|
|block_54|2014|     2459.588999199998|
+--------+----+----------------------+
only showing top 20 rows



**max energy consumption of each block for each year**

In [21]:
spark.sql('''SELECT 
                  information_df.file,
                  MAX(daily_df.energy_max) AS MAX_Energy_Consumption,
                  YEAR(daily_df.day) AS year
              FROM daily_df
              INNER JOIN information_df
              ON daily_df.LCLid = information_df.LCLid
              GROUP BY
                  information_df.file,
                  year
              ORDER BY
                  information_df.file,
                  MAX_Energy_Consumption DESC
              ''').show()

+--------+----------------------+----+
|    file|MAX_Energy_Consumption|year|
+--------+----------------------+----+
|block_22|             5.7490001|2013|
|block_22|             5.4320002|2012|
|block_22|             4.6680002|2014|
|block_22|             4.4559999|2011|
|block_35|             6.6999998|2012|
|block_35|                 6.171|2013|
|block_35|             4.6259999|2011|
|block_35|             4.5619998|2014|
|block_36|             6.5279999|2013|
|block_36|             6.5100002|2014|
|block_36|                 6.395|2012|
|block_36|                 4.829|2011|
|block_52|                 4.836|2013|
|block_52|             4.6799998|2014|
|block_52|                 4.605|2012|
|block_52|                 1.596|2011|
|block_54|    5.0209999000000005|2013|
|block_54|             4.6719999|2012|
|block_54|             3.8789999|2014|
|block_54|                 2.513|2011|
+--------+----------------------+----+
only showing top 20 rows



In [22]:
max_energy_df = spark.sql('''SELECT
                  ROW_NUMBER()OVER(PARTITION BY information_df.file ORDER BY MAX(daily_df.energy_max) DESC) as sr_no,
                  information_df.file,
                  MAX(daily_df.energy_max) AS MAX_Energy_Consumption,
                  YEAR(daily_df.day) AS year
              FROM daily_df
              INNER JOIN information_df
              ON daily_df.LCLid = information_df.LCLid
              GROUP BY
                  information_df.file,
                  year
              ORDER BY
                  information_df.file,
                  MAX_Energy_Consumption DESC
              ''')

max_energy_df.createOrReplaceTempView('max_energy_df')

max_energy_df.filter('sr_no ="1"').show()

+-----+--------+----------------------+----+
|sr_no|    file|MAX_Energy_Consumption|year|
+-----+--------+----------------------+----+
|    1|block_22|             5.7490001|2013|
|    1|block_35|             6.6999998|2012|
|    1|block_36|             6.5279999|2013|
|    1|block_52|                 4.836|2013|
|    1|block_54|    5.0209999000000005|2013|
|    1|block_62|             4.1890001|2013|
|    1|block_70|    4.2309999000000005|2013|
|    1|block_76|             6.3920002|2014|
|    1|block_87|             5.6430001|2013|
|    1|block_92|             4.9850001|2013|
|    1|block_96|             4.4899998|2012|
+-----+--------+----------------------+----+



In [23]:
halfH_df = spark.read.csv('/content/half_hourly_dataset', header=True, inferSchema=True)
halfH_df.show(5)

+---------+--------------------+--------------+
|    LCLid|                tstp|energy(kWh/hh)|
+---------+--------------------+--------------+
|MAC000002|2012-10-12 00:30:...|            0 |
|MAC000002|2012-10-12 01:00:...|            0 |
|MAC000002|2012-10-12 01:30:...|            0 |
|MAC000002|2012-10-12 02:00:...|            0 |
|MAC000002|2012-10-12 02:30:...|            0 |
+---------+--------------------+--------------+
only showing top 5 rows



In [24]:
weather_hourly_df.show(5)

+----------+-----------+-----------+-------------------+--------+--------+-------------------+---------+----------+-------------------+--------+-------------+
|visibility|windBearing|temperature|               time|dewPoint|pressure|apparentTemperature|windSpeed|precipType|               icon|humidity|      summary|
+----------+-----------+-----------+-------------------+--------+--------+-------------------+---------+----------+-------------------+--------+-------------+
|      5.97|        104|      10.24|2011-11-11 00:00:00|    8.86| 1016.76|              10.24|     2.77|      rain|partly-cloudy-night|    0.91|Partly Cloudy|
|      4.88|         99|       9.76|2011-11-11 01:00:00|    8.83| 1016.63|               8.24|     2.95|      rain|partly-cloudy-night|    0.94|Partly Cloudy|
|       3.7|         98|       9.46|2011-11-11 02:00:00|    8.79| 1016.36|               7.76|     3.17|      rain|partly-cloudy-night|    0.96|Partly Cloudy|
|      3.12|         99|       9.23|2011-11-11

In [25]:
weather_hourly_df.describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+-------------------+------------------+----------+---------+-------------------+--------------------+
|summary|        visibility|       windBearing|      temperature|         dewPoint|          pressure|apparentTemperature|         windSpeed|precipType|     icon|           humidity|             summary|
+-------+------------------+------------------+-----------------+-----------------+------------------+-------------------+------------------+----------+---------+-------------------+--------------------+
|  count|             21165|             21165|            21165|            21165|             21152|              21165|             21165|     21165|    21165|              21165|               21165|
|   mean|11.166485235058063|195.68589652728562|10.47148594377513|6.530501299314915|1014.1251527042363|   9.23033782187569|3.9052152137963905|      null|     null| 0.7818294353886075|  

In [26]:
weather_hourly_df.groupby('icon').count().show()

+-------------------+-----+
|               icon|count|
+-------------------+-----+
|                fog|  693|
|  partly-cloudy-day| 7658|
|          clear-day| 1110|
|             cloudy|  979|
|        clear-night| 4661|
|               wind|  989|
|partly-cloudy-night| 5075|
+-------------------+-----+



In [27]:
weather_hourly_df.groupby('summary').count().show()

+--------------------+-----+
|             summary|count|
+--------------------+-----+
|              Breezy|  195|
|  Windy and Overcast|    9|
|               Foggy|  693|
|Breezy and Partly...|  267|
|       Partly Cloudy| 6638|
|               Clear| 5771|
|       Mostly Cloudy| 6095|
| Breezy and Overcast|   83|
|Breezy and Mostly...|  385|
|Windy and Partly ...|    3|
|               Windy|   18|
|Windy and Mostly ...|   29|
|            Overcast|  979|
+--------------------+-----+



In [28]:
weather_hourly_df.groupby('precipType').count().show()

+----------+-----+
|precipType|count|
+----------+-----+
|      rain|20370|
|      snow|  795|
+----------+-----+



In [29]:
hh_energy_consumption_df = weather_hourly_df.withColumn('day_time',date_format('time', 'HH:mm:ss'))
hh_energy_consumption_df = hh_energy_consumption_df.withColumn('date',date_format(col('time'),"yyyy-MM-dd"))
hh_energy_consumption_df = hh_energy_consumption_df.drop('time')
hh_energy_consumption_df.show()
#hh_energy_consumption_df.createOrReplaceTempView('hh_energy_consumption_df')

+----------+-----------+-----------+--------+--------+-------------------+---------+----------+-------------------+--------+-------------+--------+----------+
|visibility|windBearing|temperature|dewPoint|pressure|apparentTemperature|windSpeed|precipType|               icon|humidity|      summary|day_time|      date|
+----------+-----------+-----------+--------+--------+-------------------+---------+----------+-------------------+--------+-------------+--------+----------+
|      5.97|        104|      10.24|    8.86| 1016.76|              10.24|     2.77|      rain|partly-cloudy-night|    0.91|Partly Cloudy|00:00:00|2011-11-11|
|      4.88|         99|       9.76|    8.83| 1016.63|               8.24|     2.95|      rain|partly-cloudy-night|    0.94|Partly Cloudy|01:00:00|2011-11-11|
|       3.7|         98|       9.46|    8.79| 1016.36|               7.76|     3.17|      rain|partly-cloudy-night|    0.96|Partly Cloudy|02:00:00|2011-11-11|
|      3.12|         99|       9.23|    8.63| 

In [30]:
col_list = weather_hourly_df.columns
print(col_list)
list_1 = ['visibility', 'windBearing', 'temperature', 'dewPoint', 'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon', 'humidity', 'summary']
weather_hourly_df.select([count(when(isnan(c),c)).alias(c) for c in list_1]).toPandas()

['visibility', 'windBearing', 'temperature', 'time', 'dewPoint', 'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon', 'humidity', 'summary']


,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
halfH_df_1 = halfH_df.withColumn('time',date_format('tstp', 'HH:mm:ss'))
halfH_df_1 = halfH_df_1.withColumn('date',date_format(col('tstp'),"yyyy-MM-dd"))
halfH_df_1 = halfH_df_1.withColumnRenamed('energy(kWh/hh)', 'energy')
halfH_df_1 = halfH_df_1.drop('tstp')
halfH_df_1.show(5)
halfH_df_1.createOrReplaceTempView('halfH_df_1')

+---------+------+--------+----------+
|    LCLid|energy|    time|      date|
+---------+------+--------+----------+
|MAC000002|    0 |00:30:00|2012-10-12|
|MAC000002|    0 |01:00:00|2012-10-12|
|MAC000002|    0 |01:30:00|2012-10-12|
|MAC000002|    0 |02:00:00|2012-10-12|
|MAC000002|    0 |02:30:00|2012-10-12|
+---------+------+--------+----------+
only showing top 5 rows



In [32]:
half_hourly_df = spark.sql(''' SELECT
                                  halfH_df_1.LCLid,
                                  halfH_df_1.energy,
                                  halfH_df_1.time,
                                  halfH_df_1.date,
                                  information_df.file
                               FROM halfH_df_1
                               INNER JOIN information_df
                               ON halfH_df_1.LCLid = information_df.LCLid
                            ''')

half_hourly_df.createOrReplaceTempView('half_hourly_df')

In [33]:
half_hourly_df.show(5)

+---------+------+--------+----------+-------+
|    LCLid|energy|    time|      date|   file|
+---------+------+--------+----------+-------+
|MAC000002|    0 |00:30:00|2012-10-12|block_0|
|MAC000002|    0 |01:00:00|2012-10-12|block_0|
|MAC000002|    0 |01:30:00|2012-10-12|block_0|
|MAC000002|    0 |02:00:00|2012-10-12|block_0|
|MAC000002|    0 |02:30:00|2012-10-12|block_0|
+---------+------+--------+----------+-------+
only showing top 5 rows



**peak hour energy consumption per day**

In [35]:
peak_hour_consumption = spark.sql(''' SELECT
                                          file,
                                          LCLid,
                                          date,
                                          SUM(energy) AS peak_hour_consumption
                                      FROM half_hourly_df
                                      WHERE time BETWEEN '09:00:00' AND '18:00:00'
                                      GROUP BY 
                                          file,
                                          LCLid,
                                          date 
                                      ORDER BY 
                                          file DESC,
                                          LCLid,
                                          date''')

peak_hour_consumption.createOrReplaceTempView('peak_hour_consumption')
peak_hour_consumption.show()

+-------+---------+----------+---------------------+
|   file|    LCLid|      date|peak_hour_consumption|
+-------+---------+----------+---------------------+
|block_9|MAC000026|2011-12-07|                5.637|
|block_9|MAC000026|2011-12-08|                 9.87|
|block_9|MAC000026|2011-12-09|    6.033999999999999|
|block_9|MAC000026|2011-12-10|   6.2780000000000005|
|block_9|MAC000026|2011-12-11|   6.6690000000000005|
|block_9|MAC000026|2011-12-12|               14.606|
|block_9|MAC000026|2011-12-13|    9.150000000000002|
|block_9|MAC000026|2011-12-14|   13.062999999999999|
|block_9|MAC000026|2011-12-15|    9.768999999999998|
|block_9|MAC000026|2011-12-16|               10.737|
|block_9|MAC000026|2011-12-17|   7.1240000000000006|
|block_9|MAC000026|2011-12-18|                7.772|
|block_9|MAC000026|2011-12-19|                9.375|
|block_9|MAC000026|2011-12-20|    5.679000099999999|
|block_9|MAC000026|2011-12-21|                3.077|
|block_9|MAC000026|2011-12-22|                

**peak hour energy consumption per month**

In [41]:
spark.sql(''' SELECT
                  file,
                  LCLid,
                  YEAR(date) AS year,
                  EXTRACT(MONTH FROM date) AS month,
                  AVG(peak_hour_consumption) AS avg_peak_hour_consumption_per_month,
                  MAX(peak_hour_consumption) AS max_peak_hour_consumption_per_month,
                  MIN(peak_hour_consumption) AS min_peak_hour_consumption_per_month
              FROM peak_hour_consumption
              GROUP BY 
                  file,
                  LCLid,
                  year,
                  month
              ORDER BY 
                  file DESC,
                  LCLid,
                  year,
                  month''').show()

+-------+---------+----+-----+-----------------------------------+-----------------------------------+-----------------------------------+
|   file|    LCLid|year|month|avg_peak_hour_consumption_per_month|max_peak_hour_consumption_per_month|min_peak_hour_consumption_per_month|
+-------+---------+----+-----+-----------------------------------+-----------------------------------+-----------------------------------+
|block_9|MAC000026|2011|   12|                        6.069640004|                             14.606|                              2.539|
|block_9|MAC000026|2012|    1|                 5.2895483903225795|                             11.083|                              2.485|
|block_9|MAC000026|2012|    2|                  6.184103458620689|                         19.0400002|                 2.7530000000000006|
|block_9|MAC000026|2012|    3|                  7.679774177419355|                         26.1049999|                              2.886|
|block_9|MAC000026|2012|   

**peak hour energy consumption per year**

In [42]:
spark.sql(''' SELECT
                  file,
                  LCLid,
                  YEAR(date) as year,
                  AVG(peak_hour_consumption) AS avg_peak_hour_consumption_per_year,
                  MAX(peak_hour_consumption) AS max_peak_hour_consumption_per_month,
                  MIN(peak_hour_consumption) AS min_peak_hour_consumption_per_month
              FROM peak_hour_consumption
              GROUP BY 
                  file,
                  LCLid,
                  year
              ORDER BY 
                  file DESC,
                  LCLid,
                  year''').show()

+-------+---------+----+----------------------------------+-----------------------------------+-----------------------------------+
|   file|    LCLid|year|avg_peak_hour_consumption_per_year|max_peak_hour_consumption_per_month|min_peak_hour_consumption_per_month|
+-------+---------+----+----------------------------------+-----------------------------------+-----------------------------------+
|block_9|MAC000026|2011|                       6.069640004|                             14.606|                              2.539|
|block_9|MAC000026|2012|                  5.86903279043716|                         26.1049999|                              2.485|
|block_9|MAC000026|2013|                  5.33105479260274|                 13.760000099999997|                 2.4070000000000005|
|block_9|MAC000026|2014|                 6.046534482758623|                         16.2480001|                              2.396|
|block_9|MAC000052|2011|                 4.679458333333334|                 

**previous and next year consumption using lag and lead**

In [60]:
spark.sql(''' SELECT
                  file,
                  LCLid,
                  YEAR(date) as year,
                  AVG(peak_hour_consumption) AS avg_peak_hour_consumption_per_year,
                  LAG(AVG(peak_hour_consumption), 1)OVER(PARTITION BY LCLid ORDER BY YEAR(date)) AS previous_year_avg_consumption,
                  LEAD(AVG(peak_hour_consumption), 1)OVER(PARTITION BY LCLid ORDER BY YEAR(date)) AS next_year_avg_consumption
              FROM peak_hour_consumption
              GROUP BY 
                  file,
                  LCLid,
                  year
              ORDER BY 
                  file DESC,
                  LCLid,
                  year''').show()

+-------+---------+----+----------------------------------+-----------------------------+-------------------------+
|   file|    LCLid|year|avg_peak_hour_consumption_per_year|previous_year_avg_consumption|next_year_avg_consumption|
+-------+---------+----+----------------------------------+-----------------------------+-------------------------+
|block_9|MAC000026|2011|                       6.069640004|                         null|         5.86903279043716|
|block_9|MAC000026|2012|                  5.86903279043716|                  6.069640004|         5.33105479260274|
|block_9|MAC000026|2013|                  5.33105479260274|             5.86903279043716|        6.046534482758623|
|block_9|MAC000026|2014|                 6.046534482758623|             5.33105479260274|                     null|
|block_9|MAC000052|2011|                 4.679458333333334|                         null|       2.5600464489071015|
|block_9|MAC000052|2012|                2.5600464489071015|            4

**year in which avg peak hour consumption is max for perticular LCLid**

In [43]:
avg_peak_hour_consumption_per_year_df = spark.sql(''' SELECT
                                                          ROW_NUMBER()OVER(PARTITION BY LCLid ORDER BY AVG(peak_hour_consumption) DESC) as sr_no,
                                                          file,
                                                          LCLid,
                                                          YEAR(date) as year,
                                                          AVG(peak_hour_consumption) AS avg_peak_hour_consumption_per_year,
                                                          MAX(peak_hour_consumption) AS max_peak_hour_consumption_per_month,
                                                          MIN(peak_hour_consumption) AS min_peak_hour_consumption_per_month
                                                      FROM peak_hour_consumption
                                                      GROUP BY 
                                                          file,
                                                          LCLid,
                                                          year
                                                      ORDER BY 
                                                          file DESC,
                                                          LCLid,
                                                          avg_peak_hour_consumption_per_year DESC,
                                                          year''')

avg_peak_hour_consumption_per_year_df.createOrReplaceTempView('avg_peak_hour_consumption_per_year_df')
avg_peak_hour_consumption_per_year_df.filter("sr_no == '1' ").show()

+-----+-------+---------+----+----------------------------------+-----------------------------------+-----------------------------------+
|sr_no|   file|    LCLid|year|avg_peak_hour_consumption_per_year|max_peak_hour_consumption_per_month|min_peak_hour_consumption_per_month|
+-----+-------+---------+----+----------------------------------+-----------------------------------+-----------------------------------+
|    1|block_9|MAC000026|2011|                       6.069640004|                             14.606|                              2.539|
|    1|block_9|MAC000052|2011|                 4.679458333333334|                 13.839999999999996|                 1.4169999999999998|
|    1|block_9|MAC000105|2011|                24.769210542105267|                         45.4920001|                             10.246|
|    1|block_9|MAC000109|2014|                 3.465120689655173|                  5.544999999999999|                 1.3299999999999998|
|    1|block_9|MAC000377|2014|    